In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load images
runoff_img = cv2.imread("/mnt/data/output2.png")
veg_img = cv2.imread("/mnt/data/output5.png")

# Resize vegetation image to match runoff image
veg_img = cv2.resize(veg_img, (runoff_img.shape[1], runoff_img.shape[0]))

# Convert BGR to RGB
runoff_img = cv2.cvtColor(runoff_img, cv2.COLOR_BGR2RGB)
veg_img = cv2.cvtColor(veg_img, cv2.COLOR_BGR2RGB)

# 1. Detect unhealthy vegetation (Red in RGB)
unhealthy_mask = np.all(veg_img == [255, 0, 0], axis=-1).astype(np.uint8)

# 2. Detect healthy vegetation (Blue)
healthy_mask = np.all(veg_img == [0, 0, 255], axis=-1).astype(np.uint8)

# 3. Detect runoff (bright areas in runoff image)
gray_runoff = cv2.cvtColor(runoff_img, cv2.COLOR_RGB2GRAY)
_, runoff_mask = cv2.threshold(gray_runoff, 50, 1, cv2.THRESH_BINARY)

# 4. Create pixel-wise classification map
# Priority: Overlap > Unhealthy > Runoff > Healthy > Background
classification = np.zeros(runoff_mask.shape, dtype=np.uint8)
classification[healthy_mask == 1] = 1
classification[unhealthy_mask == 1] = 2
classification[runoff_mask == 1] = 3
classification[(unhealthy_mask == 1) & (runoff_mask == 1)] = 4  # Overlap

# Total visible area
visible_mask = ((veg_img != [0, 0, 0]).any(axis=-1) | (gray_runoff > 10)).astype(np.uint8)
total_area = np.count_nonzero(visible_mask)

# Risky areas = unhealthy or runoff or both
risk_area = np.count_nonzero((classification == 2) | (classification == 3) | (classification == 4))

# Calculate % and level
percentage = (risk_area / total_area) * 100 if total_area else 0

if percentage <= 10:
    level = "Very Low"
elif percentage <= 30:
    level = "Low"
elif percentage <= 50:
    level = "Moderate"
elif percentage <= 70:
    level = "High"
else:
    level = "Severe"

print(f"✅ Agricultural Risk Classification:")
print(f"   Total Area Analyzed: {total_area} pixels")
print(f"   Risk Area: {risk_area} pixels")
print(f"   ⚠️ Percentage Risk: {percentage:.2f}% → Level: {level}")

# Make a colored risk map
final_map = np.zeros_like(runoff_img)
final_map[classification == 1] = [0, 0, 255]       # Healthy → Blue
final_map[classification == 2] = [255, 0, 0]       # Unhealthy → Red
final_map[classification == 3] = [0, 255, 255]     # Runoff → Cyan
final_map[classification == 4] = [255, 0, 255]     # Overlap → Magenta

# Display
plt.figure(figsize=(10, 6))
plt.title(f"🌾 Final Agricultural Risk Map → {level} ({percentage:.2f}%)")
plt.imshow(final_map)
plt.axis('off')
plt.show()


AttributeError: 'NoneType' object has no attribute 'shape'